In [9]:
import pandas as pd
import ast
import pickle

## Define each positions prime playing years as defined by The Athletic

In [2]:
goalkeeper = ['GK']
cbs_cdms_strikers = ['CB','CDM','CF', 'ST']
full_back_or_center_mid= ['LWB', 'LB', 'RB', 'RWB', 'CM']
wide_attacker_and_cam = ['LF', 'LW','LM','CAM','RF','RW','RM']
all_positions = [goalkeeper, cbs_cdms_strikers, full_back_or_center_mid, wide_attacker_and_cam]

goalkeeper_prime = list(range(27,30))
cbs_cdms_strikers_prime = list(range(26,29))
full_back_or_center_mid_prime = list(range(24,27))
wide_attacker_and_cam_prime = list(range(25,28)) 



## Define player outcome for each player

In [3]:

games = ['10', '11', '12', '13', '14', '15', '16','17', '18', '19', '20', '21']
list_of_dataframes = []

for game in games:
    df = pd.read_csv(f"fifa_{game}_top300_potential_players.csv")
    list_of_dataframes.append((game,df))
    
lived_to_expectations = 'lived to expectations'
exceeded_expectations = 'exceeded their potential'
flopped = 'did not hit their potential'
still_young = 'still has time'

list_of_dataframes = []
all_fifas = [f'Fifa {i} overall_rating' for i in range(10, 23)]
for every_fifa, x in zip(range(10, 22),range(0,13)):
    player_dataset = pd.read_csv(f'fifa_{every_fifa}_top300_potential_players.csv')
    player_outcome = []
    for i in range(player_dataset.index.start, 300):
        position = ast.literal_eval(player_dataset['position'][i])[0]
        if position in cbs_cdms_strikers:
            prime_years = cbs_cdms_strikers_prime
        elif position in full_back_or_center_mid:
            prime_years = full_back_or_center_mid_prime
        elif position in wide_attacker_and_cam:
            prime_years = wide_attacker_and_cam_prime
        else:
            prime_years = goalkeeper_prime
        potential_rating = player_dataset[f'Fifa {every_fifa} Potential rating'][i]
        for each_fifa in all_fifas:
            all_fifa_ratings = list(player_dataset[all_fifas[x:]].values[i])
            if max(all_fifa_ratings) >= potential_rating +2:
                outcome = exceeded_expectations
                break
            elif not max(all_fifa_ratings) >= potential_rating +2 and (max(all_fifa_ratings)<= potential_rating +1 and  max(all_fifa_ratings)>= potential_rating -1):
                outcome = lived_to_expectations
                break
            elif player_dataset['Fifa 22 age'][i] < min(prime_years) or player_dataset['Fifa 22 age'][i] == None:
                outcome = still_young
                break
            else:
                outcome = flopped
        player_outcome.append(outcome)
    player_dataset = pd.concat([player_dataset, pd.Series(player_outcome, name=f'Fifa {every_fifa} outcome')], axis=1)
    list_of_dataframes.append((every_fifa,player_dataset))
    exec(f'fifa_{every_fifa}_top300_potential_players = player_dataset')

## FIND THE FUNCTION YEAR POTENTIAL WAS HIT AND APPEND IT AS A NEW COLUMN IN EACH DATASET

In [4]:

def find_year_when_potential_hit(dataset,fifa_year, player_index):
    '''This function takes in a dataset, what fifa year and which player index to tell you how many years
    it took for a player to hit their potential'''
    
    all_fifa_ratings = list(dataset[all_fifas[0:]].values[player_index])
    potential_rating = dataset[f'Fifa {fifa_year} Potential rating'][player_index]
    year_potential_was_hit =[]
    for each_rating in range(potential_rating-1, 100):
        try: 
            all_fifa_ratings.index(each_rating)
        except ValueError:
            continue
        year_potential_was_hit.append(all_fifa_ratings.index(each_rating))
    try:
        min(year_potential_was_hit)  
    except ValueError:
        return None
    return min(year_potential_was_hit)


In [5]:
#intialize a list to hold the new cleaned up dataframes
list_of_dataframes = []

#iterate through each fifa
for every_fifa in range(10, 22):
    exec(f'player_dataset = fifa_{every_fifa}_top300_potential_players ')
    year_potential_hit = []
    all_fifas = [f'Fifa {i} overall_rating' for i in range(every_fifa, 23)]
    
    #iterate through each player index
    for i in range(player_dataset.index.start, 300):
        year = find_year_when_potential_hit(player_dataset, every_fifa, i)
        year_potential_hit.append(year)
    player_dataset = pd.concat([player_dataset, pd.Series(year_potential_hit, name='Year potential reached')], axis=1)
    list_of_dataframes.append((every_fifa,player_dataset))
    exec(f'fifa_{every_fifa}_top300_potential_players = player_dataset')


,Player name,Known as,player_id,position,nationality,team,Fifa 10 Potential rating,Fifa 10 overall_rating,Fifa 10 age,Fifa 11 overall_rating,...,Fifa 19 overall_rating,Fifa 19 age,Fifa 20 overall_rating,Fifa 20 age,Fifa 21 overall_rating,Fifa 21 age,Fifa 22 overall_rating,Fifa 22 age,Fifa 10 outcome,Year potential reached
0,Lionel Messi,L. Messi,158023,['RW'],Argentina,FC Barcelona,95,90,22,90.0,...,94.0,31.0,94.0,32.0,93.0,33.0,93.0,34.0,lived to expectations,2.0
1,Alexandre Pato,Alexandre Pato,180175,['CF'],Brazil,AC Milan,92,84,19,83.0,...,80.0,28.0,NaN,NaN,NaN,NaN,75.0,31.0,did not hit their potential,NaN
2,Bojan,Bojan,176993,['CF'],Spain,FC Barcelona,92,79,18,79.0,...,75.0,27.0,74.0,28.0,73.0,29.0,71.0,30.0,did not hit their potential,NaN
3,Hugo Lloris,H. Lloris,167948,['GK'],France,Olympique Lyonnais,92,83,22,84.0,...,88.0,31.0,88.0,32.0,87.0,33.0,87.0,34.0,did not hit their potential,NaN
4,Yoann Gourcuff,Y. Gourcuff,153296,['CAM'],France,FC Girondins de Bordeaux,92,85,22,83.0,...,75.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,did not hit their potential,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Bruno Pereirinha,Bruno Pereirinha,176031,['RM'],Portugal,Sporting CP,84,76,21,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,did not hit their potential,NaN
296,Kevin Prince Boateng,K. Boateng,173909,['CM'],Germany,Portsmouth,84,75,22,78.0,...,81.0,31.0,79.0,32.0,78.0,33.0,78.0,34.0,lived to expectations,3.0
297,Adrian,Adrian,173818,['ST'],Spain,Deportivo de La Coruña,84,73,21,74.0,...,77.0,30.0,76.0,31.0,75.0,32.0,NaN,NaN,did not hit their potential,NaN
298,Mark Randall,M. Randall,173538,['CM'],England,Arsenal,84,71,19,66.0,...,62.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,did not hit their potential,NaN


## Save each cleaned dataset as a csv file

In [12]:
for each_fifa, i in zip(range(10,22), range(0,12)):
    list_of_dataframes[i][1].to_csv(f'cleaned_fifa_{each_fifa}_top300_potential_players.csv',index = False)